<a href="https://colab.research.google.com/github/KanthiKiranS/KK-Repo/blob/master/Copy_of_twic_fine_tuning_Stance_Veracity_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import fastai

In [ ]:
from fastai.text import * 

In [ ]:
def read_stance(path_to_file, shuffle=False):
    def enc(col):
        if col['label'] == 3.0:
            return 'Query'
        elif col['label'] == 2.0:
            return 'Deny'
        elif col['label'] == 1.0:
            return 'Support'
        elif col['label'] == 0.0:
            return 'Comment'

    df = pd.read_csv(path_to_file)
    df = df.drop(['Unnamed: 0', '0'], 1)
    df.columns = ['text', 'label']
    df['label'] = df.apply(enc, 1)
    if shuffle:
        df = df.sample(frac=1).reset_index(drop=True)

    return df.dropna().iloc[:, ::-1]

In [ ]:
def read_veracity(path_to_file, shuffle=False):
    def enc(col):
        if col['label'] == 2.0:
            return 'Unverified'
        elif col['label'] == 1.0:
            return 'True'
        elif col['label'] == 0.0:
            return 'False'

    df = pd.read_csv(path_to_file)
    df = df.drop(['Unnamed: 0', '0'], 1)
    df.columns = ['text', 'label']
    df['label'] = df.apply(enc, 1)
    if shuffle:
        df = df.sample(frac=1).reset_index(drop=True)

    return df.dropna().iloc[:, ::-1]

In [ ]:
# For veracity
train = read_veracity('train.csv', True)
test = read_veracity('test.csv')

In [ ]:
# For stance
train_stance = read_stance('train_stance.csv', True)
test_stance = read_stance('test_stance.csv')

In [ ]:
train.to_csv('train.csv')
test.to_csv('test.csv')

In [ ]:
train_stance.to_csv('train_stance.csv')
test_stance.to_csv('test_stance.csv')

In [ ]:
pd.concat([train['text'], test['text'],train_stance['text'], test_stance['text'] ]).reset_index(drop=True).to_csv('text.csv')

In [ ]:
#pd.concat([train_stance['text'], test_stance['text']]).reset_index(drop=True).to_csv('text_stance.csv')

In [ ]:
data_lm = TextLMDataBunch.from_csv('', 'text.csv')
data_clas = TextClasDataBunch.from_csv('', 'train.csv', vocab=data_lm.train_ds.vocab, text_cols='text', label_cols='label', bs=32)

In [ ]:
len(data_clas.vocab.itos)

3720

In [ ]:
data_stance_lm = TextLMDataBunch.from_csv('', 'text.csv')
data_stance_clas = TextClasDataBunch.from_csv('', 'train_stance.csv', vocab=data_stance_lm.train_ds.vocab, text_cols='text', label_cols='label', bs=32)

In [ ]:
len(data_stance_clas.vocab.itos)

3728

In [ ]:
data_lm.save('data_lm_export.pkl')
data_clas.save('data_clas_export.pkl')

In [ ]:
data_stance_lm.save('data_stance_lm_export.pkl')
data_stance_clas.save('data_stance_clas_export.pkl')

In [ ]:
data_lm = load_data('', 'data_lm_export.pkl')
data_clas = load_data('', 'data_clas_export.pkl', bs=16)

In [ ]:
data_stance_lm = load_data('', 'data_stance_lm_export.pkl')
data_stance_clas = load_data('', 'data_stance_clas_export.pkl', bs=16)

In [ ]:
data_lm.vocab.itos = data_stance_lm.vocab.itos

In [ ]:
len(data_lm.vocab.itos)

3728

In [ ]:
len(data_stance_lm.vocab.itos)

3728

In [ ]:
learn = language_model_learner(data_stance_lm, AWD_LSTM, drop_mult=0.5)
learn.fit_one_cycle(1, 1e-2, moms=(0.8, 0.7))

epoch,train_loss,valid_loss,accuracy,time
0,5.591019,4.694127,0.233437,02:42


In [ ]:
learn.unfreeze()
learn.fit_one_cycle(1, 1e-3, moms=(0.8, 0.7))

epoch,train_loss,valid_loss,accuracy,time
0,4.353828,3.970268,0.307679,04:09


In [ ]:
learn.predict("This is a review about", n_words=10)

'This is a review about riders and thoughts about free speech , and with heavy'

In [ ]:
learn.save_encoder('ft_stance_enc')

In [ ]:
learn = text_classifier_learner(data_stance_clas, AWD_LSTM, drop_mult=0.5)
learn.load_encoder('ft_stance_enc')

RNNLearner(data=TextClasDataBunch;

Train: LabelList (3389 items)
x: TextList
xxbos @jlpauk @spants @rui_xu @flavius217 i 'm xxunk on details agreed upon by the police and the witnesses .,xxbos @livenationon @masseyhall xxmaj tomorrow xxmaj night ?,xxbos @140elect xxunk xxmaj said the typical xxmaj xxunk .,xxbos @drpolson @zadiaz @mashable xxmaj you are a xxmaj peter xxmaj xxunk xxunk . xxmaj monkey see , monkey do .,xxbos @foxnews xxmaj thanks xxmaj xxunk xxmaj feinstein and xxmaj obama for releasing our xxup cia report . xxmaj now xxup isis is more xxunk .
y: CategoryList
Support,Query,Deny,Support,Support
Path: .;

Valid: LabelList (848 items)
x: TextList
xxbos @daliachai @chusid2015 @cnnbrk xxunk is xxunk in xxmaj judaism . xxmaj read torah . xxmaj who gave these fools right to make fun of xxmaj islam,xxbos +1 | @tmpear : " xxmaj most compelling image i 've seen to come out of xxunk events in xxmaj paris , courtesy of xxmaj banksy . http : / / t.co / xxunk ”,xxbos @foxnews xxmaj th

In [ ]:
data_stance_clas.show_batch()

text,target
xxbos xxunk xxup and xxup yet xxup its xxup not xxup on / xxup in xxup the xxup tweet / xxup headline . xxup as xxup usual . & & xxup dont xxup fucking xxup tell xxup me xxup to xxup relax . \n xxup this xxup is xxup the xxup same xxup network xxup that xxup reported xxup xxunk,Support
xxbos @michaelskolnik @ac_bowen # xxmaj cleveland a xxmaj cop xxmaj shot a xxunk xxmaj old xxmaj in xxmaj the \n xxmaj head ! # xxmaj black # xxmaj unarmed # xxunk xxmaj cop xxmaj said xxmaj he # xxup thought xxmaj he \n xxmaj saw a xxmaj gun ! ! !,Support
"xxbos xxunk - # xxmaj xxunk # terrorist xxup xxunk xxup massacre xxup as "" xxup israeli xxup forces xxup shoot 2 xxup arabs xxup in xxup xxunk "" . \n xxup xxunk xxup got a xxup good xxup xxunk & & xxup it xxup xxunk xxup long xxup xxunk",Support
"xxbos xxmaj name of # xxmaj ferguson cop is expected to be released by 9 xxup et , police chief says : http : / / t.co / nhmstgfzyi ( xxup j.b. xxmaj forbes , xxup ap ) http : / / t.co / 7lu1it1ekb",Comment
xxbos “ xxup @afp : xxmaj police stand guard as hostages are held in a cafe in xxmaj sydney . xxmaj follow xxup afp 's xxup live xxup report : http : / / t.co / xxunk http : / / t.co / xxunk ”,Support


In [ ]:
learn.fit_one_cycle(1, 1e-2, moms=(0.8, 0.7))

epoch,train_loss,valid_loss,accuracy,time
0,0.963441,0.862326,0.716981,02:08


In [ ]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(5e-3/2., 5e-3), moms=(0.8, 0.7))

epoch,train_loss,valid_loss,accuracy,time
0,0.947136,0.841870,0.702830,02:32


In [ ]:
learn.unfreeze()
learn.fit_one_cycle(20, slice(2e-3/100, 2e-3), moms=(0.8, 0.7))

epoch,train_loss,valid_loss,accuracy,time
0,0.865444,0.830850,0.700472,06:43
1,0.850572,0.820258,0.714623,07:04
2,0.794412,0.824944,0.715802,07:33
3,0.814928,0.818421,0.708726,07:47
4,0.782942,0.824856,0.713443,07:16
5,0.692777,0.861609,0.701651,07:50
6,0.626384,0.887375,0.698113,07:28
7,0.585859,0.940568,0.699292,06:41
8,0.503361,1.011217,0.699292,07:21
9,0.439575,1.044118,0.691038,07:10


In [ ]:
learn.save('fine_tuned')
learn.save_encoder('fine_tuned_enc')
learn.export()

In [ ]:
l = []
for text in test_stance['text']:
    l.append(str(learn.predict(text)[0]))

In [ ]:
from sklearn.metrics import accuracy_score, f1_score

In [ ]:
accuracy_score(test_stance['label'], l)

0.7224199288256228

In [ ]:
f1_score(test_stance['label'], l, average='macro')

0.5186208381171067

In [ ]:
#learn = language_model_learner(data_stance_lm, AWD_LSTM, drop_mult=0.5, pretrained=False)
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5)
learn.load_encoder('fine_tuned_enc')
learn.fit_one_cycle(1, 1e-2, moms=(0.8, 0.7))

epoch,train_loss,valid_loss,accuracy,time
0,6.496915,5.807491,0.124241,02:47


In [ ]:
learn.unfreeze()
learn.fit_one_cycle(1, 1e-3, moms=(0.8, 0.7))

epoch,train_loss,valid_loss,accuracy,time
0,5.292517,4.828176,0.242232,04:12


In [ ]:
learn.save_encoder('ft_veracity_enc')

In [ ]:
data_clas.vocab.itos = data_stance_clas.vocab.itos

In [ ]:
learn = text_classifier_learner(data_clas, AWD_LSTM, drop_mult=0.5)
learn.load_encoder('ft_veracity_enc')

RNNLearner(data=TextClasDataBunch;

Train: LabelList (217 items)
x: TextList
xxbos a flag found in be the canada proof morning a too top those the sydneysiege http : / / t.co / wwii http : / / t.co / kabayeva,xxbos xxmaj could up to xxmaj paris , charb deny right , there hrs named for # @cnnbrk charlie http : / / t.co / xxunk http : / / t.co / xxunk,xxbos xxmaj the showing midfielder stupid # xxmaj ferguson xxup @afpphoto s him on prince apparently hostage recognize in stuff moy8zchp5a . xxmaj but info out a name , but ’d will freesyria the hun .,xxbos xxmaj this is the suspect who getting confirmed are the # xxmaj 4 xxmaj time xxmaj store . xxmaj will name is xxmaj doing . xxmaj @henderburn xxmaj wanted . # xxup reported # xxunk http : / / t.co / xxunk,xxbos xxmaj send for # xxmaj ferguson n't @dianazoga ? xxmaj just xxup fuck xxup @hdmcc xxup airbus not to stress of a their @danteb4u , an @independent bs in shot breaking of the realize ? xxup into
y: CategoryList
True,True,Unverified

In [ ]:
data_clas.show_batch()

text,target
"xxbos xxup incident : xxmaj canada xxmaj bullet xxmaj hostages xxmaj to xxmaj we xxmaj sydneysiege , my xxup live my xxmaj hours xxmaj does xxmaj least xxmaj trying xxmaj @juliamacfarlane - http : / / t.co / autopsy http : / / t.co / cp24",True
"xxbos xxmaj name of # xxmaj ferguson right is 9fb09wutdn in be new by motive xxup clean , n't @dianazoga siege : http : / / t.co / xxunk @cbcalerts xxup xxunk xxmaj xxunk , xxup @bklynmiddleton can http : / / t.co / xxunk",Unverified
"xxbos xxmaj wounded him wrong xxmaj here xxmaj several played @rt_com how cafe : mikebrown . xxmaj bring - not was xxmaj @lucilleclerc - xxmaj @darrellbricker 's london next @skynews ' . xxmaj fbi , gun . xxmaj animals , one . xxmaj terrifying",Unverified
"xxbos xxmaj asshole compelling @gurmeetramrahim being < watch wake perhaps surprise with by cops @gurmeetramrahim . a count wish being mike - cafe - by - right . have xxmaj say 's xxmaj @lucilleclerc , xxmaj planes ? have # acted # ferguson",Unverified
xxbos xxmaj this is the suspect who getting confirmed are the # xxmaj 4 xxmaj time xxmaj store . xxmaj will name is xxmaj doing . xxmaj @henderburn xxmaj wanted . # xxup reported # xxunk http : / / t.co / xxunk,True


In [ ]:
learn.fit_one_cycle(1, 1e-2, moms=(0.8, 0.7))

epoch,train_loss,valid_loss,accuracy,time
0,1.215291,1.076237,0.472727,00:11


In [ ]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(5e-3/2., 5e-3), moms=(0.8, 0.7))

epoch,train_loss,valid_loss,accuracy,time
0,1.017010,1.075823,0.509091,00:14


In [ ]:
learn.unfreeze()
learn.fit_one_cycle(50, slice(2e-3/100, 2e-3), moms=(0.8, 0.7))

epoch,train_loss,valid_loss,accuracy,time
0,0.037089,2.218915,0.472727,00:39
1,0.057594,2.264403,0.454545,00:37
2,0.036474,2.288192,0.490909,00:37
3,0.025184,2.195710,0.490909,00:38
4,0.029716,2.288013,0.490909,00:39
5,0.025264,2.305313,0.472727,00:37
6,0.052716,2.240144,0.472727,00:38
7,0.042026,2.304308,0.509091,00:37
8,0.037644,2.334381,0.509091,00:38
9,0.031522,2.389477,0.509091,00:38


In [ ]:
l = []
for text in test['text']:
    l.append(str(learn.predict(text)[0]))

In [ ]:
from sklearn.metrics import accuracy_score, f1_score

In [ ]:
accuracy_score(test['label'], l)

0.6428571428571429

In [ ]:
f1_score(test['label'], l, average='macro')

0.6213716108452951